# Retail sales order total regression

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
# Ignore all warnings
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [ ]:
# !pip install openpyxl

### Reading data and initial EDA

In [ ]:
df = pd.read_excel('/kaggle/input/retail-insights-a-comprehensive-sales-dataset/data.xlsx')

In [ ]:
df.info()

In [ ]:
# Count plot for product categories
plt.figure(figsize=(12, 8))
sns.countplot(x='Product Category', data=df, palette='viridis')
plt.title("Product Category Distribution")
plt.show()

# Count plot for order priority
plt.figure(figsize=(10, 6))
sns.countplot(x='Order Priority', data=df, palette='muted')
plt.title("Order Priority Distribution")
plt.show()

# Count plot for ship mode
plt.figure(figsize=(10, 6))
sns.countplot(x='Ship Mode', data=df, palette='pastel')
plt.title("Ship Mode Distribution")
plt.show()

### Data Preprocessing

In [ ]:
columns_to_drop = ['Order No', 'City','State','Customer Name', 'Address']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

In [ ]:
# Drop rows with missing values in 'Address' and 'Order Quantity'
df.dropna(subset=['Order Quantity'], inplace=True)

df['Order day'] = df['Order Date'].dt.weekday
df['Ship day'] = df['Ship Date'].dt.weekday
df['Order Month'] = df['Order Date'].dt.month
df['Ship Month'] = df['Ship Date'].dt.month
# One-hot encoding for categorical columns
df = pd.get_dummies(df, columns=['Ship day','Order day','Customer Type','Account Manager','Order Priority','Product Category','Product Container','Ship Mode'])

columns_to_convert = ['Cost Price', 'Retail Price', 'Profit Margin', 'Sub Total', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']

for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col].str.replace('$', ''), errors='coerce')

# Drop the specified columns
df = df.drop(columns=['Order Date','Product Name','Ship Date'])
df.dropna(subset=list(df.columns), inplace=True)



### Scaling Data

In [ ]:
df.info()

In [ ]:
numeric_columns = ['Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']
correlation_matrix = df[numeric_columns].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap for Numeric Columns")
plt.show()

In [ ]:
df[numeric_columns].hist(figsize=(15, 10), bins=20, edgecolor='k', alpha=0.7)
plt.suptitle("Distribution of Numeric Columns", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df[numeric_columns])
plt.suptitle("Pairplot for Numeric Columns", y=1.02)
plt.show()

In [ ]:
scaler = StandardScaler()
numerical_columns = ['Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost', 'Total']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

X = df.drop('Total',axis=1)
y = df['Total']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Cross-validation
cv_mse = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2').mean()

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'Cross-validated MSE: {cv_mse}')
print(f'Cross-validated R-squared: {cv_r2}')

In [ ]:
original_columns = ['Cost Price', 'Retail Price', 'Profit Margin', 'Order Quantity', 'Sub Total', 'Discount %', 'Discount $', 'Order Total', 'Shipping Cost']

# Make sure X_test has the same columns as original_data
X_test_subset = X_test[original_columns]
# Concatenate X_test_subset and y_pred
concatenated_data = pd.concat([X_test_subset, pd.Series(y_pred, name='Total')], axis=1)

# Inverse transform the concatenated data
newdf_original_scale = pd.DataFrame(scaler.inverse_transform(concatenated_data), columns=concatenated_data.columns)

In [ ]:
from sklearn.svm import SVR

# Training Support Vector Regression (SVR) model
svr_model = SVR(kernel='linear')
svr_model.fit(X_train, y_train)
# Making predictions on the test set
y_pred_svr = svr_model.predict(X_test)
# Evaluating the model
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)
cv_mse_svr = cross_val_score(svr_model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
cv_r2_svr = cross_val_score(svr_model, X, y, cv=5, scoring='r2').mean()
print("\nSupport Vector Regression (SVR) Model:")
print(f'Mean Squared Error: {mse_svr}')
print(f'R-squared: {r2_svr}')
print(f'Cross-validated MSE: {cv_mse_svr}')
print(f'Cross-validated R-squared: {cv_r2_svr}')

In [ ]:
from sklearn.linear_model import Ridge
# Training Ridge Regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
# Making predictions on the test set
y_pred_ridge = ridge_model.predict(X_test)
# Evaluating the model
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
cv_mse_ridge = cross_val_score(ridge_model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
cv_r2_ridge = cross_val_score(ridge_model, X, y, cv=5, scoring='r2').mean()
print("\nRidge Regression Model:")
print(f'Mean Squared Error: {mse_ridge}')
print(f'R-squared: {r2_ridge}')
print(f'Cross-validated MSE: {cv_mse_ridge}')
print(f'Cross-validated R-squared: {cv_r2_ridge}')